In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
original_dataset = pd.read_csv('data.csv')
original_dataset = original_dataset
original_dataset.describe()

### Preprocessing

**Drop transactions with value 0**

In [ ]:
dataset = original_dataset
dataset = dataset.drop(dataset[dataset['Amt'] == 0].index)
dataset = dataset.drop(dataset[dataset['Buyer'] == dataset['Seller']].index)
#dataset = dataset[:200000]
dataset.describe()

### Performing benford analysis on the overall transactions

In [ ]:
from math import log10

def benford_analysis(vals):
    # Expected Benford frequencies
    expected_freq = [round(len(vals) * log10(1 + 1/d)) for d in range(1, 10)]

    # Convert to scientific notation and get first digit
    first_digit = lambda x: int(('%e' % x)[0])

    # Get observed first digit frequencies
    observed_freq = [0] * 10
    for val in vals:
        observed_freq[first_digit(val)] += 1
    observed_freq = observed_freq[1:]

    print('Expected frequencies: ', expected_freq)
    print('Observed frequencies: ', observed_freq)
    
    plt.plot(range(1, 10), expected_freq, label='Expected')
    plt.plot(range(1, 10), observed_freq, label='Observed')
    plt.legend(loc='upper right')
    plt.show()
    
    mean_abs_dev = 1/(len(vals)*9) * sum([abs(obv-exp)
                                          for obv, exp in zip(observed_freq, expected_freq)])
    print('Mean absolute deviation: %.6lf' % mean_abs_dev)

In [ ]:
benford_analysis(dataset['Amt'])

$\text{Mean Absolute Deviation (MAD) is around 0.017 which implies a nonconformity between the expected probability and} \\ \text{the observed probability.}$

### Pruning the transaction graph

- We only consider the nodes (users) which have both incoming and outgoing edges.
- This is because our goal is to identify circular trading and nodes need to have both incoming and outgoing edges to be part of a cycle.

In [ ]:
sellers = set(dataset['Seller'])
buyers = set(dataset['Buyer'])
seller_buyer_union = sellers.union(buyers)
seller_buyer_intersection = sellers.intersection(buyers)

print('Sellers:', len(sellers))
print('Buyers:', len(buyers))
print('Sellers Union Buyers:', len(seller_buyer_union))
print('Sellers Intersection Buyers:', len(seller_buyer_intersection))

In [ ]:
import networkx as nx

In [ ]:
G = nx.DiGraph()

for seller, buyer, amt in dataset[['Seller', 'Buyer', 'Amt']].values:
    if (seller in seller_buyer_intersection
            and buyer in seller_buyer_intersection):
        G.add_weighted_edges_from([(int(seller), int(buyer), amt)])

In [ ]:
# To access edges with weights set data=True
#G.edges(data=True)

In [ ]:
## Betweenness Clustering

from networkx.algorithms.community.centrality import girvan_newman

clusters = girvan_newman(G)

In [ ]:
#tuple(sorted(c) for c in next(clusters))

#cluster = tuple(sorted(c) for c in next(clusters))

In [ ]:
# print("Number of clusters: ", len(cluster))

# print(cluster[1])

# for i in G.edges:
#     if i[0] in cluster[1]  and i[1] in cluster[1]:
#         print(i)

In [ ]:
#HCS

#edges = nx.minimum_edge_cut(G)

# UG1 = nx.Graph(G)
# UG = UG1

# print("Number of components: ", nx.number_connected_components(UG1))

# comp = nx.connected_components(UG1)

# cnt = 0

# for i in comp:
#     UG = UG1.subgraph(i).copy()
#     cnt += 1
#     if cnt > 0:
#         break
    
# print("Len: ", len(UG.nodes()))

In [ ]:
# Return sets of nodes after running HCS
# Graph should be connected at first

def HCS(G):
    edges = nx.minimum_edge_cut(G)
    
    #print("Big bum")
    
    num_nodes = len(G.nodes());
    if (num_nodes < 3) or (len(edges) > num_nodes/2) :
        #print("Nodes: ", G.nodes())
        return [list(G.nodes())]
    
    for i in edges:
        #print(i)
        G.remove_edge(i[0], i[1])
    
    num_comps = nx.number_connected_components(G)
#     if  num_comps < 2:
#         print()
#         return [sorted(list(G.nodes()))]
    
    comp = nx.connected_components(G)
    
    graphs = []
    for i in comp:
        g = G.subgraph(i).copy()
        graphs += HCS(g)
        
    return graphs

In [ ]:
#print(HCS(UG.copy()))

In [ ]:
# Weights of Triangles

def get_weighted_graph(G):
    
    weights = []
    
    print("Num edges: ", len(G.edges()))
    
    edge_set = {}
    
    for i in range(len(G.edges())):
        weights.append([0,0,0,0])
    
    for i, edge in enumerate(G.edges()):
        u = edge[0]
        v = edge[1]
        
        suc_set = set(list(G.successors(v)))
        pred_set = set(list(G.predecessors(u)))
        
        common = suc_set.intersection(pred_set)
        
        for j in common:
            cnt = 0
            if G.has_edge(v,u):
                cnt+=1
            if G.has_edge(j,v):
                cnt+=1
            if G.has_edge(u,j):
                cnt+=1
                
            weights[i][cnt] = cnt+1
            
            if u < v:
                edge_set[(u,v)] = max(edge_set.get((u,v),0),cnt+1)
            else :
                edge_set[(v,u)] = max(edge_set.get((v,u),0),cnt+1)
            
            #print("u: ", u, ", v: ", v, ", j: ", j)
        
    ans_graph = nx.Graph()

    #for i, edge in enumerate(G.edges()):
    #    ans_graph.add_weighted_edges_from([(edge[0], edge[1], max(weights[i]))])
    
    for (u,v),w in edge_set.items():
        ans_graph.add_weighted_edges_from([(u, v, w)]) 
        
    return ans_graph

In [ ]:
#tmp = nx.complete_graph(10)
# tmp = nx.DiGraph()
# tmp.add_edge(1,2)
# tmp.add_edge(2,1)
# tmp.add_edge(2,3)
# tmp.add_edge(3,1)
# tmp.add_edge(1,4)
# tmp.add_edge(4,5)
# tmp.add_edge(5,1)
# tmp.add_edge(5,6)

tmp = G

tmp = get_weighted_graph(tmp)

# #tmp.edges(data=True)
print("Edges: ", len(tmp.edges()))
print("Nodes: ", len(tmp.nodes()))

In [ ]:
## kNN filter
def get_k_n(G, n, k):
    edges = list(G.out_edges(n, data=True))
    
    edges.sort(key = lambda x : x[-1]['weight'], reverse=True)
    
    return edges[:k]
    #print(edges[0])

def knn_filter(G, k):
    node_list = list(G.nodes())
    
    ans_graph = nx.DiGraph()
    
    for i in node_list:
        edges = get_k_n(G, i, k)
        ans_graph.add_weighted_edges_from( [(edge[0], edge[1], edge[-1]['weight']) for edge in edges] )
        
    return ans_graph


In [ ]:
tmp = nx.DiGraph()
tmp.add_weighted_edges_from([(1,2,10), (1,3,12), (1,4,10), (2,3,7), (2,4,4), (3,1,6), (3,4,3), (3,5,100)])
# tmp.add_edge(2,1, weight=10)
# tmp.add_edge(2,3, weight=10)
# tmp.add_edge(3,1, weight=10)
# tmp.add_edge(1,4, weight=10)
# tmp.add_edge(4,5, weight=10)
# tmp.add_edge(5,1, weight=10)
# tmp.add_edge(5,6, weight=10)

tmp = knn_filter(tmp, 2)
tmp.edges(data=True)

In [ ]:
# Collusion clustering
def is_km_compatible(G, p, d_size, m):
    if len(G.edges(p)) >= min(m, d_size):
        return True
    return False

def is_kmh_compatible(G, s1, s2, m, h):
    
    s1_percent = 0
    for i in s1:
        if is_km_compatible(G, i, len(s2), m) :
            s1_percent += 1
    s1_percent /= len(s1)
    
    s2_percent = 0
    for i in s2:
        if is_km_compatible(G, i, len(s1), m) :
            s2_percent += 1
    s2_percent /= len(s2)
    
    if s1_percent >= h and s2_percent >= h:
        return True
    return False

def collusion_index(G, s):
    i_c = 0
    e_c = 0
    
    for edge in G.edges(data=True):
        if edge[0] in s and edge[1] in s:
            i_c += edge[-1]['weight']
            continue
            
        if edge[0] in s or edge[1] in s:
            e_c += edge[-1]['weight']
            
    if e_c == 0:
        return 100
        
    return i_c/e_c

def collusion_level(G, s1, s2):
    s = s1.intersection(s2)
    return collusion_index(G, s)

# Make sure 1 <= m <= k
def collusion_clustering(G, k, m, h):
    
    G = knn_filter(G, k)
    S = []
    for i in G.nodes():
        S.append({i})
        
    #print(S)
    
    while True:
        B = []
        for i, s_i in enumerate(S):
            for j in range(i+1,len(S)):
                s_j = S[j]
                B.append((s_i, s_j, collusion_level(G, s_i, s_j)))
    
        B.sort(key = lambda x : x[-1], reverse=True)
        
        no_change = True
        
        for i in B:
            if (i[0] not in S) or (i[1] not in S):
                continue
                
            if i[-1] > 0 and is_kmh_compatible(G, i[0], i[1], m, h) :
                no_change = False
                S.remove(i[0])
                S.remove(i[1])
                
                new_set = i[0].union(i[1])
                
                S.append(new_set)
                
        if no_change:
            break
            
    return S
    

In [ ]:
#tmp = nx.DiGraph()
#tmp.add_weighted_edges_from([(1,2,10), (1,3,12), (1,4,10), (2,3,7), (2,4,4), (3,1,6), (3,4,3), (3,5,100)])

tmp = get_weighted_graph(G)
#sets = collusion_clustering(tmp, 2, 1, 0.5)
comp = nx.connected_components(tmp)

for i in comp:
    tmp = tmp.subgraph(i).copy()
    break

sets = HCS(tmp)

print("Len: ", len(sets))
print(sets)
